# Functional Patterns: A Python "Ramda" Cookbook

This notebook explores functional programming patterns for common data manipulation tasks, particularly for those coming from a JavaScript/TypeScript background with experience using libraries like Ramda or Lodash.

Python's functional capabilities are often built directly into the language or provided by libraries like `returns`. We'll explore how to achieve the same declarative style in a way that is idiomatic to Python.

## 1. Declarative Iteration: `map`, `filter`, and Comprehensions

A core tenet of functional programming is to express *what* you want to do, not *how* you want to do it. Instead of writing imperative `for` loops, we use declarative approaches like `map` and `filter`.

### Style 1: Pipeline-based (Ramda-like)

Using `returns.pipe` with `map` and `filter` feels very similar to Ramda's `R.pipe`. This style is excellent for long chains of transformations.

In [ ]:
from returns.pipeline import pipe

numbers = [1, 2, 3, 4, 5, 6]

# Goal: Take the even numbers, square them, and get a list of the results.

result_pipe = pipe(
    lambda nums: filter(lambda x: x % 2 == 0, nums),
    lambda evens: map(lambda x: x * x, evens),
    list,  # Materialize the iterators into a list
)(numbers)

# --- Verification ---
assert result_pipe == [4, 16, 36]
print(f"✅ Pipeline correctly transformed the list to {result_pipe}")

### Style 2: List Comprehensions (The "Pythonic" Way)

For many simple to moderately complex cases, a list comprehension is often considered more readable and idiomatic in Python. It combines the `map` and `filter` operations into a single, expressive statement.

In [ ]:
numbers = [1, 2, 3, 4, 5, 6]

# The same goal, achieved with a list comprehension:
result_comprehension = [x * x for x in numbers if x % 2 == 0]

# --- Verification ---
assert result_comprehension == [4, 16, 36]
print(f"✅ List comprehension correctly transformed the list to {result_comprehension}")

## 2. Currying & Partial Application

Currying is the process of transforming a function that takes multiple arguments into a sequence of functions that each take a single argument. This is incredibly useful for creating specialized functions from more general ones.

The `returns` library provides a `@curry` decorator that makes this easy. A best practice when designing functions for currying is to put the "data" argument last.

In [ ]:
from returns.curry import curry


@curry
def add(x: int, y: int) -> int:
    return x + y


# --- Verification ---

# 1. Call it normally
assert add(3, 4) == 7

# 2. Create a specialized function by partially applying the first argument
add_five = add(5)
assert callable(add_five)  # It's a new function!

# 3. Call the specialized function
assert add_five(10) == 15
assert add_five(20) == 25

print("✅ Currying works as expected, creating specialized functions.")

## 3. Logical Operations: `allPass` / `anyPass`

Ramda provides useful functions like `allPass` (returns `True` if a value satisfies all predicates) and `anyPass` (returns `True` if it satisfies any). We can easily create these helpers in Python.

This demonstrates a key principle: when a helper function is missing, write a small, pure, and reusable function yourself.

In [ ]:
from collections.abc import Callable
from typing import TypeVar

T = TypeVar("T")
Predicate = Callable[[T], bool]


def all_pass[T](predicates: list[Predicate[T]]) -> Predicate[T]:
    """Creates a new function that checks if a value passes all predicates."""

    def checker(value: T) -> bool:
        return all(p(value) for p in predicates)

    return checker


# --- Verification ---


# Define some predicates for checking a number
def is_even(x: int) -> bool:
    return x % 2 == 0


def is_positive(x: int) -> bool:
    return x > 0


def is_less_than_100(x: int) -> bool:
    return x < 100


# Create a combined validator
is_valid_number = all_pass([is_even, is_positive, is_less_than_100])

# Test cases
assert is_valid_number(10)
assert not is_valid_number(11)
assert not is_valid_number(-2)
assert not is_valid_number(102)

print("✅ `all_pass` helper correctly combines predicates.")